This notebook moves towards GPy and GPyOpt libraries to make use of their GP and Bayesian optimization implementations (the main goal is still to show seasonality breaks standard methods)

In [ ]:
import math
import numpy as np
import GPy
import GPyOpt
import time

import sys
sys.path.append("../../PyDeepGP")
import deepgp

import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

In [ ]:
x = np.linspace(0, 1, 1000)
plt.plot(x, np.sin(2 * math.pi * np.linspace(-0 * 1.0 / 7, 1 - 0 * 1.0 / 7, 1000)), label="0")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-1 * 1.0 / 7, 1 - 1 * 1.0 / 7, 1000)), label="1")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-2 * 1.0 / 7, 1 - 2 * 1.0 / 7, 1000)), label="2")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-3 * 1.0 / 7, 1 - 3 * 1.0 / 7, 1000)), label="3")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-4 * 1.0 / 7, 1 - 4 * 1.0 / 7, 1000)), label="4")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-5 * 1.0 / 7, 1 - 5 * 1.0 / 7, 1000)), label="5")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-6 * 1.0 / 7, 1 - 6 * 1.0 / 7, 1000)), label="6")
plt.title("Example of a moving periodic signal")
plt.legend()
plt.show()

In [ ]:
class DataGenerator:
    def __init__(self, sigma_obs):
        self.day_of_the_week = 0
        self.sigma_obs = sigma_obs

    def sample(self, x):
        # Handling cases when x is both scalar and numpy array
        self.day_of_the_week = (self.day_of_the_week + 1) % 7
        return np.sin(2 * math.pi * (x - self.day_of_the_week * 1.0 / 7)) + self.sigma_obs * np.random.randn()


## Confirming basic Bayesian optimization breaks

In [ ]:
data_gen = DataGenerator(sigma_obs=0.1)

def f(parameters):
    parameters = parameters[0]
    x = parameters[0]
    score = -data_gen.sample(x)
    score = np.array(score)
    return score

In [ ]:
bounds = [
            {'name': 'x', 'type': 'continuous', 'domain': (0, 1)}
         ]

In [ ]:
np.random.seed(777)
optimizer = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds,
                                                acquisition_type ='MPI',
                                                acquisition_par = 0.1,
                                                exact_eval=True)

In [ ]:
max_iter = 50
max_time = 60
optimizer.run_optimization(max_iter, max_time)

In [ ]:
plt.scatter(optimizer.X[:, 0], optimizer.Y)
plt.title("Sampled points during standard Bayesian optimization")
plt.show()

## GP without seasonality knowledge

In [ ]:
kernel = GPy.kern.RBF(input_dim=1)
model = GPy.models.GPRegression(optimizer.X, optimizer.Y, kernel)
model.optimize()

model.plot()
plt.show()

## Trying 2D ARD kernels to account for seasonality

In [ ]:
class DataGenerator:
    def __init__(self, sigma_obs):
        self.day_of_the_week = 0
        self.sigma_obs = sigma_obs

    def sample(self, x):
        # Handling cases when x is both scalar and numpy array
        self.day_of_the_week = (self.day_of_the_week + 1) % 7
        signal = np.sin(2 * math.pi * (x - self.day_of_the_week * 1.0 / 7)) + self.sigma_obs * np.random.randn()
        return [self.day_of_the_week, x, signal]
    
    def generate_random_sample(self, n_obs):
        obs = []
        for _ in range(n_obs):
            obs.append(self.sample(np.random.random()))
        return np.array(obs)

In [ ]:
np.random.seed(123456)
data_gen = DataGenerator(sigma_obs=0.1)
sample = data_gen.generate_random_sample(50)

With ARD=False the multidimensional kernel is the basic 1D kernel for the norm of the difference between vectors

With ARD=True the multidimensional kernel is product of differently scaled 1D kernels for elements of vectors

In [ ]:
kernel = GPy.kern.RBF(input_dim=2, ARD=True)
model = GPy.models.GPRegression(sample[:, :2], sample[:, 2].reshape(-1, 1), kernel)
model.optimize()

model.plot()
plt.show()

In [ ]:
x = np.linspace(0, 1, 1000)

plt.figure(figsize=(10, 25))
plt.title("Example of a moving periodic signal")

for i in range(7):
    plt.subplot(7, 1, i + 1)
    x_pred = np.hstack([i * np.ones(x.shape).reshape(-1, 1), x.reshape(-1, 1)])
    pred = model.predict(x_pred)
    plt.plot(x, pred[0], label=str(i) + "_infered")   
    plt.plot(x, np.sin(2 * math.pi * (x - i * 1.0 / 7)), label=str(i) + "_true")  
    plt.plot(sample[np.where(sample[:, 0] == i)][:, 1], sample[np.where(sample[:, 0] == i)][:, 2], ".", color="r", label="samples")
    plt.fill_between(x, (pred[0] - 2 * pred[1]).reshape(-1), (pred[0] + 2 * pred[1]).reshape(-1), color="b", alpha=0.25)    
    plt.legend()
    
plt.show()

In [ ]:
kernel.plot_ARD()

## Fitting DeepGP

In [ ]:
kern1 = GPy.kern.RBF(2, ARD=True) + GPy.kern.Bias(2)
kern2 = GPy.kern.RBF(2, ARD=True) + GPy.kern.Bias(2)

model = deepgp.DeepGP(nDims=[1, 2, 2],
                      Y=sample[:, 2].reshape(-1, 1),
                      X=sample[:, :2],
                      kernels=[kern1,kern2], 
                      num_inducing=10, 
                      back_constraint=False
                     )

In [ ]:
model.optimize(max_iters=5000, messages=True)

In [ ]:
x = np.linspace(0, 1, 1000)

plt.figure(figsize=(10, 25))
plt.title("Example of a moving periodic signal")

for i in range(7):
    plt.subplot(7, 1, i + 1)
    x_pred = np.hstack([i * np.ones(x.shape).reshape(-1, 1), x.reshape(-1, 1)])
    pred = model.predict(x_pred)
    plt.plot(x, pred[0], label=str(i) + "_infered")   
    plt.plot(x, np.sin(2 * math.pi * (x - i * 1.0 / 7)), label=str(i) + "_true")  
    plt.plot(sample[np.where(sample[:, 0] == i)][:, 1], sample[np.where(sample[:, 0] == i)][:, 2], ".", color="r", label="samples")
    plt.fill_between(x, (pred[0] - 2 * pred[1]).reshape(-1), (pred[0] + 2 * pred[1]).reshape(-1), color="b", alpha=0.25)    
    plt.legend()
    
plt.show()

In [ ]:
model.obslayer.kern.plot_ARD()
model.layer_1.kern.plot_ARD()